In [1]:
import random
import syft as sy
from syft.core import utils
from syft.core.frameworks.torch import utils as torch_utils
from syft.core.frameworks import encode
from syft.core.frameworks.torch.tensor import _MPCTensor
from syft.core.frameworks.torch.tensor import _GeneralizedPointerTensor
from syft.mpc import spdz
import torch
import torch.nn.functional as F
from torch.autograd import Variable as Var
import json

hook = sy.TorchHook(verbose=True)

me = hook.local_worker
me.is_client_worker = False

bob = sy.VirtualWorker(id="bob", hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice", hook=hook, is_client_worker=False)
james = sy.VirtualWorker(id="james", hook=hook, is_client_worker=False)

bob.add_workers([alice, james])
alice.add_workers([bob, james])
james.add_workers([bob, alice])

In [2]:
x = torch.LongTensor([1,2,3,4])

In [3]:
x.send(bob)

LongTensor[_PointerTensor - id:1709951101 owner:0 loc:bob id@loc:4248161797]

In [4]:
x.get_shape()

torch.Size([4])

In [5]:
encoded = x._encode()

In [7]:
encoded.shape

torch.Size([])

In [6]:
encoded.get_shape()

torch.Size([4])

In [5]:
location = bob
n_workers = 2
secret = encoded
mod = spdz.field

In [6]:
random_shares = [torch.LongTensor(secret.shape) for i in range(n_workers - 1)]

for share in random_shares:
    if(location is not None):
        share.send(location)
    share.random_(mod)


In [9]:
secret.shape

torch.Size([])

In [9]:
shares = list()

In [10]:
shares.append(random_shares[0])

In [11]:
share = secret - random_shares[0]

In [ ]:
shares = []
for i in range(n_workers):
    if i == 0:
        share = random_shares[i]
    elif i < n_workers - 1:
        share = random_shares[i] - random_shares[i-1]
    else:
        share = secret - random_shares[i-1]
    shares.append(share)


In [6]:
# shared = x._share(2)

LongTensor[_PointerTensor - id:6803938345 owner:0 loc:bob id@loc:5393764004]